## Bases de datos no convenciales



## Contenidos
* [Ejercicio 1. Listado de todas las publicaciones de un autor determinado](#ej1)
* [Ejercicio 2. Número de publicaciones de un autor determinado](#ej2)
* [Ejercicio 3. Número de artículos en revista para el año 2016](#ej3)
* [Ejercicio 4. Número de autores ocasionales, es decir, que tengan menos de 5 publicaciones en total](#ej4)
* [Ejercicio 5. Número de artículos de revista (article) y número de artículos en congresos (inproceedings) de los diez autores con más publicaciones totales](#ej5)
* [Ejercicio 6. Número medio de autores de todas las publicaciones que tenga en su conjunto de datos](#ej6)
* [Ejercicio 7. Listado de coautores de un autor (Se denomina coautor a cualquier persona que haya firmado una publicación)](#ej7)
* [Ejercicio 8. Edad de los 5 autores con un periodo de publicaciones más largo (Se considera la Edad de un autor al número de años transcurridos desde la fecha de su primera publicación hasta la última registrada)](#ej8)
* [Ejercicio 9. Número de autores novatos, es decir, que tengan una Edad menor de 5 años (Se considera la edad de un autor al número de años transcurridos desde la fecha de su primera publicación hasta la última registrada)](#ej9)
* [Ejercicio 10. Porcentaje de publicaciones en revistas con respecto al total de publicaciones](#ej10)

Importación de librerias e instaciación del cliente Mongo

In [2]:
from pymongo import MongoClient
from pprint import pprint

# Mongo ready on ECS instance, set the proper ip
mongo_server_host = "176.34.147.136"

client = MongoClient("mongodb://" + mongo_server_host + ":27017")
db = client['bdnc']

def printCursorContent(cursor,n=0):
    cnt=1
    for val in cursor:
        if cnt==n:
            return
        else:
            pprint(val)
        cnt = cnt + 1

In [3]:
publicaciones =  db.dblp

### Listado de todas las publicaciones de un autor determinado.  <a class="anchor" id="ej1"></a>

In [4]:
author="E. F. Codd"
print("Listado de publicaciones de {0}".format(author))
c = publicaciones.find({"Authors":author}, {"item": 1, "status": 1, "title": 1, '_id': 0})
printCursorContent(c)

Listado de publicaciones de E. F. Codd
{u'title': u'Seven Steps to Rendezvous with the Casual User.'}
{u'title': u'Interactive Support for Non-Programmers: The Relational and Network Approaches.'}
{u'title': u'The Capabilities of Relational Database Management Systems.'}
{u'title': u'Relational Completeness of Data Base Sublanguages.'}
{u'title': u'Further Normalization of the Data Base Relational Model.'}
{u'title': u'Data Base Sublanguage Founded on the Relational Calculus.'}
{u'title': u'Derivability, Redundancy and Consistency of Relations Stored in Large Data Banks.'}
{u'title': u'Normalized Data Base Structure: A Brief Tutorial.'}
{u'title': u'Fatal Flaws in SQL - Part Two.'}
{u'title': u'Fatal Flaws in SQL - Part ONE.'}
{u'title': u'Extending the Database Relational Model to Capture More Meaning.'}
{u'title': u"'Universal' Relation Fails to Replace Relational Model (letter to the editor)."}
{u'title': u'Multiprogramming.'}
{u'title': u'Understanding Relations (Third Installment)

### Número de publicaciones de un autor determinado<a class="anchor" id="ej2"></a>

In [5]:
author="E. F. Codd"
print("Número de publicaciones de {0}".format(author))
c = publicaciones.aggregate([{"$match":{"Authors":author}},\
                             {"$group":{"_id":"null", "count":{"$sum":1}}}])
printCursorContent(c)

Número de publicaciones de E. F. Codd
{u'_id': u'null', u'count': 34}


### Número de artículos en revista para el año 2016<a class="anchor" id="ej3"></a>

In [6]:
pubtype = 'article'
year = 2016
print("Número de {0} en el año {1}".format(pubtype, year))
c = publicaciones.aggregate([{"$match":{"year": year, "type": pubtype}}, \
                             {"$group":{"_id":{'year': '$year', 'type': '$pubtype'}, "count":{"$sum":1}}}])
printCursorContent(c)

Número de article en el año 2016
{u'_id': {u'year': 2016}, u'count': 127165}


### Número de autores ocasionales, es decir, que tengan menos de 5 publicaciones en total<a class="anchor" id="ej4"></a>

In [7]:
print("Número de autores ocasionales")
c = publicaciones.aggregate([{'$unwind': "$Authors" }, \
                             {'$sortByCount': "$Authors"}, \
                             {'$match': {'count':{'$lt':5} } }, \
                             {'$count': "Autores Ocasionales"} ], \
                            allowDiskUse=True)
printCursorContent(c)

Número de autores ocasionales
{u'Autores Ocasionales': 1103389}


### Número de artículos de revista (article) y número de artículos en congresos (inproceedings) de los diez autores con más publicaciones totales<a class="anchor" id="ej5"></a>

In [10]:
print("Número de articulos de revista y congresos de \
los de autores con más publicaciones totales")
c = publicaciones.aggregate([{'$match': { '$or': [ {'type':"inproceedings"}, \
                                              {'type':"article"} ] } }, \
                             {'$unwind': "$Authors" }, \
                             {'$group': {'_id':{ 'Authors': "$Authors"}, \
                                        'publicaciones': {'$sum':1}, \
                                        "type": {"$push":"$type"} }}, \
                             {'$sort': {"publicaciones":-1}}, \
                             {'$limit':10}, \
                             {'$unwind': "$type" }, \
                             {'$group': {'_id': {'type': "$type", \
                                             'Authors': "$_id.Authors"}, \
                                         
                                        'count': {'$sum':1} } }, \
                             {'$sort': {"_id.Authors":1}}], allowDiskUse=True)
printCursorContent(c)

Número de articulos de revista y congresos de los de autores con más publicaciones totales
{u'_id': {u'Authors': u'Chin-Chen Chang', u'type': u'inproceedings'},
 u'count': 21}
{u'_id': {u'Authors': u'Chin-Chen Chang', u'type': u'article'}, u'count': 584}
{u'_id': {u'Authors': u'H. Vincent Poor', u'type': u'article'}, u'count': 915}
{u'_id': {u'Authors': u'H. Vincent Poor', u'type': u'inproceedings'},
 u'count': 57}
{u'_id': {u'Authors': u'Jing Li', u'type': u'article'}, u'count': 406}
{u'_id': {u'Authors': u'Jing Li', u'type': u'inproceedings'}, u'count': 131}
{u'_id': {u'Authors': u'Jun Liu', u'type': u'article'}, u'count': 440}
{u'_id': {u'Authors': u'Jun Liu', u'type': u'inproceedings'}, u'count': 83}
{u'_id': {u'Authors': u'Lajos Hanzo', u'type': u'article'}, u'count': 558}
{u'_id': {u'Authors': u'Lajos Hanzo', u'type': u'inproceedings'}, u'count': 63}
{u'_id': {u'Authors': u'Mohamed-Slim Alouini', u'type': u'inproceedings'},
 u'count': 57}
{u'_id': {u'Authors': u'Mohamed-Slim Alou

### Número medio de autores de todas las publicaciones que tenga en su conjunto de datos<a class="anchor" id="ej6"></a>

In [11]:
print("Número medio de autores")
c = publicaciones.aggregate([{'$unwind': "$Authors" }, \
                             {'$group': {'_id': '$_id', 'count':{'$sum':1}}},
                             {'$group': {'_id':'null', 'media':{'$avg':'$count'}}}], \
                            allowDiskUse=True)
printCursorContent(c)

Número medio de autores
{u'_id': u'null', u'media': 2.799367417887333}


### Listado de coautores de un autor (Se denomina coautor a cualquier persona que haya firmado una publicación)<a class="anchor" id="ej7"></a>

In [12]:

author="E. F. Codd"
print("Listado de coatuores de {0}".format(author))
c = publicaciones.aggregate([{"$match":{"Authors":author}},\
                             {"$unwind": '$Authors'}, \
                             {'$group': {'_id': '$Authors', 'count':{'$sum':1}}}, \
                             {'$match':{'_id':{'$ne':author}}}, 
                             {'$project': {'_id':1}}], \
                           allowDiskUse=True)
printCursorContent(c)

Listado de coatuores de E. F. Codd
{u'_id': u'Casper A. Scalzi'}
{u'_id': u'C. J. Date'}
{u'_id': u'E. S. Lowry'}
{u'_id': u'E. McDonough'}


### Edad de los 5 autores con un periodo de publicaciones más largo (Se considera la Edad de un autor al número de años transcurridos desde la fecha de su primera publicación hasta la última registrada)<a class="anchor" id="ej8"></a>

In [13]:
print("Autores con periodo de publicaciones más largo")
c = publicaciones.aggregate([{'$unwind': "$Authors" } , \
                             {'$group': {'_id': '$Authors', \
                                        'anyos': {'$push': '$year'}}}, \
                             {'$project': {'maximo': {'$max': "$anyos"}, \
                                        'minimo':{ '$min': "$anyos"}}}, \
                             {'$project': {'edad': {'$subtract': ["$maximo", \
                                                            "$minimo"]}}}, \
                             {'$sort': {'edad':-1}}, \
                             {'$limit': 5}], \
                            allowDiskUse=True)
                            
printCursorContent(c)

Autores con periodo de publicaciones más largo
{u'_id': u'Alan M. Turing', u'edad': 75}
{u'_id': u'Rudolf Carnap', u'edad': 71}
{u'_id': u'David Nelson', u'edad': 64}
{u'_id': u'Eric Weiss', u'edad': 64}
{u'_id': u'George E. Collins', u'edad': 63}


### Número de autores novatos, es decir, que tengan una Edad menor de 5 años (Se considera la edad de un autor al número de años transcurridos desde la fecha de su primera publicación hasta la última registrada)<a class="anchor" id="ej9"></a>

In [14]:
print("Numero de autores novatos")
c = publicaciones.aggregate([{'$unwind': "$Authors" } , \
                             {'$group' : { '_id': '$Authors', \
                                          'anyos': { '$push': "$year"} }}, \
                             {'$project':{ 'maximo': {'$max': "$anyos"}, \
                                          'minimo':{ '$min': "$anyos"} }}, \
                             {'$project': {'edad': {'$subtract': ["$maximo", "$minimo"] } } }, \
                             {'$match': {'edad': {'$lt':5}}}, \
                             {'$count': "AutoresNovatos"} ], \
                             allowDiskUse=True)
printCursorContent(c)

Numero de autores novatos
{u'AutoresNovatos': 1021673}


### Porcentaje de publicaciones en revistas con respecto al total de publicaciones<a class="anchor" id="ej10"></a>

In [15]:
print("Porcentaje de publicaciones en revista")
c = publicaciones.aggregate([{'$match': { 'type': "article"} }, \
                   {'$group': {'_id':'$type', 'count':{'$sum':1}}}, \
                   {'$project':{'_id':0, \
                              'Porcentaje':{ '$multiply': [{'$divide': [100, \
                                                                        publicaciones.count()]}, \
                                                             "$count"]}}}],
                           allowDiskUse = True)
printCursorContent(c)

Porcentaje de publicaciones en revista
{u'Porcentaje': 79.71885}
